In [2]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v2.0 - CORREGIDO
El bug estaba en el orden de bytes del Merkle Root
"""

import hashlib
import struct
import socket
import time
import random
from typing import List, Dict, Tuple, Optional

class KaoruBridgeV2:
    """Versión corregida con el Merkle Root en orden correcto."""

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    DNS_SEEDS = [
        'seed.bitcoin.sipa.be',
        'dnsseed.bluematt.me',
        'seed.bitcoinstats.com',
        'seed.bitcoin.jonasschnelli.ch',
    ]

    def __init__(self):
        self.magic = self.MAINNET_MAGIC

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def construct_genesis_header(self) -> bytes:
        """
        Construye el Genesis Block header CORRECTAMENTE.

        ⚠️ EL BUG ESTABA AQUÍ - El merkle root debe estar en little-endian
        """
        version = struct.pack('<I', 1)
        prev_hash = bytes(32)

        # CORRECCIÓN: El merkle root en el bloque está en little-endian
        # El valor que vemos normalmente está en big-endian (human readable)
        merkle_root_be = "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b"
        merkle_root = bytes.fromhex(merkle_root_be)[::-1]  # Convertir a little-endian

        timestamp = struct.pack('<I', 1231006505)
        bits = struct.pack('<I', 0x1d00ffff)
        nonce = struct.pack('<I', 2083236893)

        return version + prev_hash + merkle_root + timestamp + bits + nonce

    def verify_genesis(self) -> Tuple[bool, Dict]:
        """Verifica que nuestro genesis coincide con el real."""
        header = self.construct_genesis_header()
        computed_hash = self.double_sha256(header)[::-1].hex()

        return computed_hash == self.GENESIS_HASH, {
            'header_hex': header.hex(),
            'computed_hash': computed_hash,
            'expected_hash': self.GENESIS_HASH,
            'match': computed_hash == self.GENESIS_HASH
        }

    def create_message(self, command: str, payload: bytes) -> bytes:
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = self.double_sha256(payload)[:4]
        return self.magic + command_bytes + length + checksum + payload

    def create_version_payload(self, remote_ip: str = '127.0.0.1') -> bytes:
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))

        addr_recv = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton(remote_ip) + struct.pack('>H', 8333)
        addr_from = struct.pack('<Q', 0) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton('0.0.0.0') + struct.pack('>H', 8333)

        nonce = struct.pack('<Q', random.randint(0, 2**64-1))
        user_agent = b'\x14/KaoruBridge:2.0.0/'
        start_height = struct.pack('<i', 0)
        relay = b'\x00'

        return (version + services + timestamp + addr_recv +
                addr_from + nonce + user_agent + start_height + relay)

    def create_headers_payload(self, headers: List[bytes]) -> bytes:
        """Crea payload para mensaje HEADERS."""
        count = len(headers)
        payload = struct.pack('B', count)  # Varint
        for header in headers:
            payload += header
            payload += b'\x00'  # tx_count = 0
        return payload

    def create_getdata_response(self, block_header: bytes, coinbase_tx: bytes) -> bytes:
        """Crea un bloque completo para responder a GETDATA."""
        tx_count = b'\x01'  # 1 transacción
        return block_header + tx_count + coinbase_tx

    def create_coinbase_tx(self, message: str = "KAORU BRIDGE") -> bytes:
        """Crea transacción coinbase."""
        tx = struct.pack('<I', 1)  # Version
        tx += b'\x01'  # Input count
        tx += bytes(32)  # Prev txid (null)
        tx += struct.pack('<I', 0xffffffff)  # Prev vout

        # ScriptSig con nuestro mensaje
        script_sig = bytes([len(message)]) + message.encode()
        tx += struct.pack('B', len(script_sig)) + script_sig
        tx += struct.pack('<I', 0xffffffff)  # Sequence

        tx += b'\x01'  # Output count
        tx += struct.pack('<Q', 50 * 10**8)  # 50 BTC

        # ScriptPubKey (OP_RETURN con mensaje)
        script_pubkey = b'\x6a' + bytes([len(message)]) + message.encode()
        tx += struct.pack('B', len(script_pubkey)) + script_pubkey

        tx += struct.pack('<I', 0)  # Locktime

        return tx

    def discover_nodes(self) -> List[Tuple[str, int]]:
        nodes = []
        for seed in self.DNS_SEEDS:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                for ip in ips[:3]:
                    nodes.append((ip, 8333))
            except:
                pass
        return nodes

    def connect(self, ip: str, port: int = 8333, timeout: int = 10) -> Optional[socket.socket]:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            sock.connect((ip, port))

            version_msg = self.create_message('version', self.create_version_payload(ip))
            sock.send(version_msg)

            response = sock.recv(1024)
            if b'version' in response:
                verack_msg = self.create_message('verack', b'')
                sock.send(verack_msg)
                sock.recv(1024)  # Recibir verack
                return sock

            sock.close()
            return None
        except:
            return None

    def execute_full_protocol(self):
        """
        PROTOCOLO COMPLETO v2.0
        Ahora con el genesis correcto + manejo de GETHEADERS
        """

        print("""
╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║     ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗                  ║
║     ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║                  ║
║     █████╔╝ ███████║██║   ██║██████╔╝██║   ██║                  ║
║     ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║                  ║
║     ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝                  ║
║     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝                  ║
║                                                                  ║
║              BRIDGE PROTOCOL v2.0 - BUG FIXED                   ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1: Verificar genesis
        print("="*70)
        print("[FASE 1] 🔍 VERIFICANDO GENESIS BLOCK (CORREGIDO)")
        print("="*70)

        success, data = self.verify_genesis()

        print(f"\n   Header (80 bytes):")
        header_hex = data['header_hex']
        print(f"   {header_hex[:64]}")
        print(f"   {header_hex[64:128]}")
        print(f"   {header_hex[128:]}")

        print(f"\n   Computed Hash: {data['computed_hash']}")
        print(f"   Expected Hash: {data['expected_hash']}")
        print(f"\n   ✅ MATCH: {data['match']}")

        if not success:
            print("\n   ❌ ERROR: El genesis aún no coincide!")
            return False

        # FASE 2: Extraer W-values correctos
        print("\n" + "="*70)
        print("[FASE 2] 🔬 W-SCHEDULE EXTRACTION (CORRECTO)")
        print("="*70)

        header = self.construct_genesis_header()

        # W-values en BIG-ENDIAN para SHA-256
        print("\n   W-Schedule (primeros 16 valores):")
        labels = ["Version", "PrevHash[0]", "PrevHash[1]", "PrevHash[2]",
                  "PrevHash[3]", "PrevHash[4]", "PrevHash[5]", "PrevHash[6]",
                  "PrevHash[7]", "MerkleRoot[0]", "MerkleRoot[1]", "MerkleRoot[2]",
                  "MerkleRoot[3]", "MerkleRoot[4]", "MerkleRoot[5]", "MerkleRoot[6]"]

        for i in range(16):
            w = struct.unpack('>I', header[i*4:(i+1)*4])[0]
            print(f"   W[{i:2d}] = 0x{w:08x}  ({labels[i]})")

        # FASE 3: Conectar a la red
        print("\n" + "="*70)
        print("[FASE 3] 🌐 CONEXIÓN A MAINNET")
        print("="*70)

        nodes = self.discover_nodes()
        print(f"\n   Nodos descubiertos: {len(nodes)}")

        sock = None
        connected_ip = None

        for ip, port in nodes[:5]:
            print(f"   Intentando {ip}:{port}...", end=" ")
            sock = self.connect(ip, port)
            if sock:
                print("✅")
                connected_ip = ip
                break
            print("❌")

        if not sock:
            print("\n   ⚠️ No se pudo conectar")
            return False

        # FASE 4: Interacción avanzada
        print("\n" + "="*70)
        print("[FASE 4] 💉 PROTOCOLO DE INYECCIÓN AVANZADO")
        print("="*70)

        print(f"""
   ┌─────────────────────────────────────────────────────────────┐
   │  CONVERSACIÓN CON EL NODO                                   │
   └─────────────────────────────────────────────────────────────┘
        """)

        # Enviar GETHEADERS pidiendo desde antes del genesis
        print("   📤 TÚ: 'Oye, ¿qué headers tienes desde el bloque 0?'")

        getheaders_payload = struct.pack('<I', 70015)  # Version
        getheaders_payload += b'\x01'  # 1 hash
        getheaders_payload += bytes(32)  # All zeros = desde el principio
        getheaders_payload += bytes(32)  # Stop hash = dame todo

        getheaders_msg = self.create_message('getheaders', getheaders_payload)
        sock.send(getheaders_msg)

        time.sleep(1)

        try:
            response = sock.recv(4096)
            if len(response) >= 24:
                cmd = response[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
                print(f"   📥 NODO: '{cmd}' - ", end="")

                if cmd == 'headers':
                    # Parsear headers
                    payload = response[24:]
                    if len(payload) > 0:
                        count = payload[0]
                        print(f"Te envío {count} headers")

                        if count > 0 and len(payload) > 81:
                            first_header = payload[1:81]
                            first_hash = self.double_sha256(first_header)[::-1].hex()
                            print(f"\n   📋 Primer header recibido:")
                            print(f"      Hash: {first_hash}")
                            print(f"      ¿Es el genesis? {first_hash == self.GENESIS_HASH}")
                else:
                    print(f"respuesta de tipo '{cmd}'")

        except socket.timeout:
            print("   ⏱️ Timeout esperando respuesta")
        except Exception as e:
            print(f"   ❌ Error: {e}")

        # FASE 5: Intentar inyectar bloque alternativo
        print("\n" + "-"*70)
        print("   📤 TÚ: 'Tengo un bloque génesis ALTERNATIVO'")

        # Crear genesis alternativo (con nuestro mensaje)
        alt_merkle = self.double_sha256(b"KAORU_BRIDGE_GENESIS_2024")

        alt_header = struct.pack('<I', 1)  # Version
        alt_header += bytes(32)  # Prev hash
        alt_header += alt_merkle  # Nuestro merkle
        alt_header += struct.pack('<I', int(time.time()))  # Timestamp actual
        alt_header += struct.pack('<I', 0x1d00ffff)  # Bits
        alt_header += struct.pack('<I', 0)  # Nonce (sin minar)

        alt_hash = self.double_sha256(alt_header)[::-1].hex()
        print(f"      Hash alternativo: {alt_hash}")

        # Anunciar con INV
        inv_payload = b'\x01'  # 1 item
        inv_payload += struct.pack('<I', 2)  # MSG_BLOCK
        inv_payload += bytes.fromhex(alt_hash)[::-1]

        inv_msg = self.create_message('inv', inv_payload)
        sock.send(inv_msg)

        time.sleep(1)

        try:
            response = sock.recv(4096)
            if len(response) >= 24:
                cmd = response[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
                print(f"   📥 NODO: '{cmd}'")

                if cmd == 'getdata':
                    print("\n   🤯 ¡¡EL NODO QUIERE NUESTRO BLOQUE!!")
                    print("   📤 Enviando bloque alternativo...")

                    coinbase = self.create_coinbase_tx("KAORU_BRIDGE_WAS_HERE")
                    full_block = alt_header + b'\x01' + coinbase
                    block_msg = self.create_message('block', full_block)
                    sock.send(block_msg)

                    time.sleep(2)

                    try:
                        final_response = sock.recv(4096)
                        if len(final_response) >= 24:
                            final_cmd = final_response[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
                            print(f"   📥 NODO: '{final_cmd}'")

                            if final_cmd == 'reject':
                                print("\n   ❌ NODO: 'Bloque rechazado - hash inválido'")
                            elif final_cmd == 'ping':
                                print("\n   🏓 NODO: 'ping' (nos ignora)")
                        else:
                            print("   📥 NODO: *silencio* (nos desconectó)")
                    except:
                        print("   📥 NODO: *nos desconectó*")

        except socket.timeout:
            print("   ⏱️ Timeout")
        except Exception as e:
            print(f"   ❌ Error: {e}")

        sock.close()

        # RESULTADO FINAL
        print("\n" + "="*70)
        print("                    📊 RESULTADO FINAL")
        print("="*70)

        print(f"""
   ┌─────────────────────────────────────────────────────────────────┐
   │                                                                 │
   │  ✅ Genesis original verificado correctamente                  │
   │  ✅ Conexión a mainnet establecida                              │
   │  ✅ Protocolo P2P funcional                                     │
   │  ✅ Mensajes enviados y procesados                              │
   │                                                                 │
   │  ❌ El nodo rechazó/ignoró nuestro bloque alternativo          │
   │                                                                 │
   │  CONCLUSIÓN:                                                    │
   │  El protocolo Bitcoin funciona exactamente como se diseñó.     │
   │  Los nodos SOLO aceptan bloques que:                           │
   │                                                                 │
   │  1. Tengan hash con suficientes ceros (proof of work)          │
   │  2. Continúen la cadena más larga                              │
   │  3. NO contradigan el genesis hardcodeado                      │
   │                                                                 │
   └─────────────────────────────────────────────────────────────────┘

   🔗 Nodo contactado: {connected_ip}
   📋 Genesis verificado: {self.GENESIS_HASH[:32]}...
   💉 Bloque alternativo: {alt_hash[:32]}...

   Estado: BRIDGE PROTOCOL COMPLETADO ✅
        """)

        return True


# ============================================
# EJECUCIÓN
# ============================================

if __name__ == "__main__":
    bridge = KaoruBridgeV2()
    bridge.execute_full_protocol()


╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║     ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗                  ║
║     ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║                  ║
║     █████╔╝ ███████║██║   ██║██████╔╝██║   ██║                  ║
║     ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║                  ║
║     ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝                  ║
║     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝                  ║
║                                                                  ║
║              BRIDGE PROTOCOL v2.0 - BUG FIXED                   ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
        
[FASE 1] 🔍 VERIFICANDO GENESIS BLOCK (CORREGIDO)

   Header (80 bytes):
   0100000000000000000000000000000000000000000000000000000000000000
   000000003ba3edfd7a7b12b27ac

No le hagan caso a la nota JAJAJJ, es que al crear el codigo luego me ponen esas notas randoms JAJAJAJ